In [1]:
#### To start we will import all of our needed libraries and dive into our datasets.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.impute import KNNImputer

from py_files.feature_engineering import filter_column_strings, knn_impute


In [2]:
df = pd.read_csv('data/post_eda_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

/tmp/ipykernel_82568/3373641402.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/post_eda_data.csv')


In [3]:
df.head()

,index,id,status_group,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,pop_height
0,0,69572,functional,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,NaN
1,1,8776,functional,NaN,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN
2,2,34310,functional,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,NaN
3,3,67743,non functional,NaN,Unicef,263.0,UNICEF,38.486161,-11.155298,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,NaN
4,4,19728,functional,NaN,Action In A,NaN,Artisan,31.130847,-1.825359,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN


## To Do:

#### String Clean up
   
#### Construction year to age
#### KNNImpute
    - population
    - gps_height
    - amount_tsh¶
    
#### Bin population

#### Mask features based on value count



In [ ]:
# String Clean up 

columns_ = [
    'funder',
    'installer',
    'scheme_name',
    'subvillage',
    'ward',
    'basin',
    'region',
    'lga',
    'ward',
    'scheme_management',
    'scheme_name',
    'extraction_type',
    'extraction_type_group',
    'extraction_type_class',
    'management',
    'management_group',
    'payment',
    'payment_type',
    'water_quality',
    'quality_group',
    'quantity',
    'quantity_group',
    'source',
    'source_type',
    'source_class',
    'waterpoint_type',
    'waterpoint_type_group'
]

for col_ in columns_:
    df[col_] = df[col_].str.replace(' ', '') \
        .str.lower() \
        .str.replace('(','') \
        .str.replace(')', '') \
        .str.replace('/', '') \
        .str.replace('_', '')
    
    filter_column_strings(90, df, col_, fuzz.token_sort_ratio, drop=True)


In [ ]:
# Construction Year To Age
df['construction_year'] = 2022 - df['construction_year']
df = df.rename(columns={'construction_year': 'well_age'})

In [ ]:
mask = df.funder.map(df.funder.value_counts()) == 1
df.funder = df.funder.mask(mask, 'low_count_funder')

In [ ]:
knn_impute(df, test_df, 'population', [0,1], 66)
knn_impute(df, test_df, 'gps_height', 0, 66)
knn_impute(df, test_df, 'amount_tsh', 0, 5)

In [ ]:
test_df = df[~df.status_group.notna()].copy()
test_df.drop('status_group', axis=1, inplace=True)
df = df[df.status_group.notna()].copy()

In [ ]:
for col_ in columns:
    if col_ != 'status_group':
        df, test_df = cat_label_pct(df, test_df, col)
        df.drop(col, axis=1, inplace=True)
        test_df.drop(col, axis=1, inplace=True)

In [ ]:
test_df.fillna(0.0, inplace=True)
df.fillna(0.0, inplace=True)

In [ ]:
# test_df.to_csv('data/test_set_imputed.txt')
# df.to_csv('data/df_imputed.txt')

df = pd.read_csv('data/df_imputed.csv')
test_df = pd.read_csv('data/test_set_imputed.csv')
df_ = df.copy()

In [ ]:
list_ = [1,1,1,2,3,4,5]

In [ ]:
# Verifying ID's are identical in the test dataset and nan values
pd.DataFrame(df.loc[df['status_group'].isna()]['id'] == test_df['id'])['id'].value_counts()